# Manitoba Tariff Extraction Pipeline

**Phase 1**: Extract tariff codes with L1/L2/L3/L4 hierarchy  
**Phase 2**: GPT enrichment for metadata extraction

**Features:**
- Extracts 4,600+ physician billing codes from Manitoba's Payment Schedule
- Hierarchical categorization (Section → Category → Subcategory)
- AI-powered enrichment: parent codes, add-ons, restrictions, exclusions
- Checkpointing for crash recovery during long runs

---

In [ ]:
!pip install openai -q
import re
import json
import time
import pandas as pd
from typing import Optional, List, Tuple, Dict
from openai import OpenAI
from google.colab import files, userdata
import getpass
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
print("✓ Imports ready")

✓ Imports ready


In [ ]:
# Upload source file
print("Upload the marked-up payment schedule text file (with L1/L2/L3 markers)")
uploaded = files.upload()
SOURCE_FILE = list(uploaded.keys())[0]
with open(SOURCE_FILE, 'r', encoding='utf-8') as f:
    RAW_TEXT = f.read()
LINES = RAW_TEXT.split('\n')
print(f"✓ {len(LINES):,} lines loaded")

Upload the marked-up payment schedule text file (with L1/L2/L3 markers)


Saving mb_payment_schedule_marked (2).txt to mb_payment_schedule_marked (2).txt
✓ 30,570 lines loaded


---
## Configuration & Constants
---

In [ ]:
# =============================================================================
# CONFIGURATION CONSTANTS
# =============================================================================

# Extraction settings
CONTENT_START_MARKER = '«L1:RULESOFAPPLICATION»'
CONTENT_START_MIN_LINE = 1000
HIERARCHY_LOOKAHEAD_LINES = 5

# Output settings
MAX_DESCRIPTION_LENGTH = 500
MAX_NOTES_LENGTH = 1000
MAX_CONTEXT_LENGTH = 3500

# GPT settings
GPT_MODEL = "gpt-5.2"
GPT_TEMPERATURE = 0
API_CALL_DELAY_SECONDS = 0.1
COST_PER_ENTRY_ESTIMATE = 0.005
TIME_PER_ENTRY_SECONDS = 0.5

# Progress reporting
PROGRESS_REPORT_INTERVAL = 25

# Checkpointing settings
CHECKPOINT_INTERVAL = 100  # Save checkpoint every N entries
CHECKPOINT_FILE = 'MB_phase2_checkpoint.json'

# Fee parsing thresholds (for distinguishing unit values from fees)
UNIT_VALUE_MIN = 19
UNIT_VALUE_MAX = 28
FEE_MIN_FOR_UNIT_DETECTION = 50

# Section code mappings
SECTION_PATTERNS = [
    (r'visit|examination|internal medicine|general practice', 'A'),
    (r'general schedule', 'B'),
    (r'anesthesia', 'C'),
    (r'integumentary|skin|breast', 'D'),
    (r'musculoskeletal', 'E'),
    (r'respiratory', 'F'),
    (r'cardiovascular', 'G'),
    (r'digestive', 'H'),
    (r'urinary', 'I'),
    (r'male genital', 'J'),
    (r'female genital|obstetric|gyn', 'K'),
    (r'maternity', 'L'),
    (r'endocrine', 'M'),
    (r'nervous', 'N'),
    (r'eye|ocular|ophthal', 'O'),
    (r'ear|otol', 'P'),
    (r'nose|nasal|rhinol', 'Q'),
    (r'diagnostic radiological', 'T'),
    (r'nuclear medicine', 'U'),
    (r'therapeutic radiological', 'V'),
    (r'laboratory', 'W'),
]

# Add-on detection patterns
ADD_ON_PATTERNS = [
    r'\badd\b', r'\badd-on\b', r'\badditional\b', r'\bsupplement\b',
    r'\beach additional\b', r'\bper additional\b', r'\badd to\b',
]

# Final output column order
OUTPUT_COLUMNS = [
    'tariff_code', 'tariff_code_display', 'parent_code',
    'section_code', 'section_name', 'specialty_code', 'specialty_name',
    'category', 'subcategory', 'subsubcategory',
    'description', 'notes',
    'fee_total', 'fee_technical', 'fee_professional', 'unit_value',
    'is_add_on', 'add_on_to',
    'age_restriction', 'setting_restriction', 'exclusions',
    'is_provisional', 'is_asterisked', 'is_by_report',
    'is_cross_reference', 'cross_reference_to',
    'applicable_rules', 'time_requirement_minutes'
]

print("✓ Configuration constants defined")

✓ Configuration constants defined


---
## Phase 1: Extraction with L1/L2/L3 Hierarchy
---

In [ ]:
# =============================================================================
# TEXT CLEANING UTILITIES
# =============================================================================

def clean_text(text: str) -> str:
    """Normalize dashes and clean encoding issues."""
    text = text.replace('—', '-')
    text = text.replace('–', '-')
    text = text.replace('−', '-')
    text = text.replace('‐', '-')
    text = text.replace('\u00a0', ' ')
    text = re.sub(r'\s+', ' ', text).strip()
    return text


def to_title_case(text: str) -> str:
    """Convert text to title case, handling special patterns."""
    if pd.isna(text) or not str(text).strip():
        return text
    text = str(text).strip()

    # Handle 'O FFICE , H OME V ISITS' pattern (spaced first letters)
    if re.match(r'^[A-Z]\s+[A-Z]', text):
        text = re.sub(r'\b([A-Z])\s+([A-Z]+)', r'\1\2', text)
        text = re.sub(r'\s*,\s*', ', ', text)
        text = re.sub(r'\s+', ' ', text)

    text = text.title()

    # Preserve lowercase for small words
    for word in ['And', 'Or', 'The', 'A', 'An', 'Of', 'In', 'For', 'To', 'By']:
        text = re.sub(r'\s' + word + r'\s', ' ' + word.lower() + ' ', text)

    return text


print("✓ Text cleaning utilities defined")

✓ Text cleaning utilities defined


In [ ]:
# =============================================================================
# SECTION AND SPECIALTY LOOKUPS
# =============================================================================

def get_section_code(l1_text: str) -> str:
    """Map L1 section text to section code (A-W)."""
    l1_lower = l1_text.lower()
    for pattern, code in SECTION_PATTERNS:
        if re.search(pattern, l1_lower):
            return code
    return ''


def get_specialty_info(l1_text: str) -> Tuple[str, str]:
    """Extract specialty code and name from L1 text."""
    match = re.search(r'(.+?)\s*\((\d{2}(?:-\d+)?)\)', l1_text)
    if match:
        name = match.group(1).strip()
        name = re.sub(r'^Visits/Examinations[—–-]\s*', '', name)
        return match.group(2), name
    return '', ''


print("✓ Section and specialty lookups defined")

✓ Section and specialty lookups defined


In [ ]:
# =============================================================================
# FEE AND CONTENT EXTRACTION
# =============================================================================

def find_content_start(lines: List[str]) -> int:
    """Find the line where main content begins."""
    for i, line in enumerate(lines):
        if CONTENT_START_MARKER in line and i > CONTENT_START_MIN_LINE:
            return i
    return 0


def parse_fee(val: str) -> float:
    """Parse fee string, handling comma-separated thousands (e.g., '1,053.59')."""
    return float(val.replace(',', ''))


def extract_fee_from_block(block: str) -> dict:
    """Parse fee information from a code block."""
    result = {'fee': None, 'tec': None, 'pro': None, 'unit_value': None, 'is_by_report': False}

    if re.search(r'By Report', block, re.IGNORECASE):
        result['is_by_report'] = True

    # Pattern for fees: handles comma-separated thousands (e.g., 1,053.59 or 112.42)
    FEE_PATTERN = r'(\d{1,3}(?:,\d{3})*\.\d{2})'
    UNIT_PATTERN = r'(\d{2}\.\d{3})'

    # Pattern: fee + unit value (e.g., "...1,053.59  25.500" or "...112.42  21.375")
    match = re.search(r'\.{3,}\s*' + FEE_PATTERN + r'\s+' + UNIT_PATTERN + r'\s*$', block, re.MULTILINE)
    if match:
        result['fee'] = parse_fee(match.group(1))
        result['unit_value'] = float(match.group(2))
        return result

    # Pattern: two decimal values (could be TEC/PRO or fee + unit)
    match = re.search(r'\.{3,}\s*' + FEE_PATTERN + r'\s+' + FEE_PATTERN + r'\s*$', block, re.MULTILINE)
    if match:
        val1, val2 = parse_fee(match.group(1)), parse_fee(match.group(2))
        if UNIT_VALUE_MIN < val2 < UNIT_VALUE_MAX and val1 > FEE_MIN_FOR_UNIT_DETECTION:
            result['fee'] = val1
            result['unit_value'] = val2
        else:
            result['tec'] = val1
            result['pro'] = val2
            result['fee'] = val1 + val2
        return result

    # Pattern: single fee (e.g., "...1,053.59" or "...112.42")
    match = re.search(r'\.{3,}\s*' + FEE_PATTERN + r'\s*$', block, re.MULTILINE)
    if match:
        result['fee'] = parse_fee(match.group(1))
        return result

    # Pattern: By Report with unit value
    match = re.search(r'By Report.*?' + UNIT_PATTERN + r'\s*$', block, re.MULTILINE | re.IGNORECASE)
    if match:
        result['unit_value'] = float(match.group(1))

    return result


def extract_description(block: str, code: str) -> str:
    """Extract and clean description text from a code block."""
    text = re.sub(r'«CODE:~?\d{4}\*?»', '', block)
    lines = []
    for line in text.split('\n'):
        if re.match(r'^\s*Notes?:', line, re.IGNORECASE):
            break
        if re.match(r'^\s*April 1,|^[A-Z]-\d+\s*$|^\f', line):
            continue
        if line.strip():
            lines.append(line)
    desc = ' '.join(lines)
    desc = re.sub(r'^\s*~?' + code + r'\*?\s*', '', desc)
    desc = re.sub(r'\s*\.{3,}.*$', '', desc)
    desc = re.sub(r'\s*By Report.*$', '', desc, flags=re.IGNORECASE)
    desc = clean_text(desc)
    return desc


def extract_notes(block: str) -> str:
    """Extract notes section from a code block."""
    match = re.search(r'Notes?:\s*(.+?)(?=«|$)', block, re.DOTALL | re.IGNORECASE)
    if match:
        notes = clean_text(match.group(1))
        return notes[:MAX_NOTES_LENGTH]
    return ''


def check_cross_reference(block: str) -> Tuple[bool, str]:
    """Check if block contains a cross-reference to another section."""
    patterns = [r'See (General Schedule|Section [A-Z])', r'-See ([A-Za-z ]+Schedule)']
    for pattern in patterns:
        match = re.search(pattern, block, re.IGNORECASE)
        if match:
            return True, match.group(1)
    return False, ''


def extract_rules(block: str) -> str:
    """Extract applicable rule references from a code block."""
    rules = set()
    for match in re.finditer(r'Rules?\s+(\d+(?:\s+to\s+\d+)?)', block, re.IGNORECASE):
        rules.add(match.group(1))
    return ', '.join(sorted(rules)) if rules else ''


def extract_time_requirement(block: str) -> Optional[int]:
    """Extract minimum time requirement in minutes from a code block."""
    match = re.search(r'minimum of.*?(\d+).*?minutes', block, re.IGNORECASE)
    return int(match.group(1)) if match else None


def is_add_on_fee(block: str, description: str) -> bool:
    """Detect if this is an add-on fee based on text patterns."""
    text = (block + ' ' + description).lower()
    for pattern in ADD_ON_PATTERNS:
        if re.search(pattern, text):
            return True
    return False


print("✓ Fee and content extraction functions defined")

✓ Fee and content extraction functions defined


In [ ]:
# =============================================================================
# HIERARCHY EXTRACTION
# =============================================================================

def extract_hierarchy_text(lines: List[str], start_idx: int, fallback_text: str) -> str:
    """
    Extract readable text for a hierarchy level marker.

    Looks ahead from the marker line to find the first non-empty,
    non-marker line and extracts the readable portion.

    Args:
        lines: All source lines
        start_idx: Index of the hierarchy marker line
        fallback_text: Text to use if no readable text found

    Returns:
        Cleaned readable text for this hierarchy level
    """
    for k in range(start_idx + 1, min(start_idx + HIERARCHY_LOOKAHEAD_LINES, len(lines))):
        next_line = lines[k].strip()
        if next_line and not next_line.startswith('«') and not next_line.startswith('\f'):
            return clean_text(next_line.split('...')[0])
    return clean_text(fallback_text)


class HierarchyTracker:
    """
    Tracks the current L1/L2/L3/L4 hierarchy state during extraction.

    When a higher level is set, all lower levels are automatically reset.
    """

    def __init__(self):
        self.l1 = ""
        self.l2 = ""
        self.l3 = ""
        self.l4 = ""

    def set_l1(self, value: str):
        """Set L1 and reset L2, L3, L4."""
        self.l1 = value
        self.l2 = ""
        self.l3 = ""
        self.l4 = ""

    def set_l2(self, value: str):
        """Set L2 and reset L3, L4."""
        self.l2 = value
        self.l3 = ""
        self.l4 = ""

    def set_l3(self, value: str):
        """Set L3 and reset L4."""
        self.l3 = value
        self.l4 = ""

    def set_l4(self, value: str):
        """Set L4."""
        self.l4 = value

    def get_current(self) -> Tuple[str, str, str, str]:
        """Return current hierarchy as tuple (l1, l2, l3, l4)."""
        return self.l1, self.l2, self.l3, self.l4


print("✓ Hierarchy extraction functions defined")

✓ Hierarchy extraction functions defined


In [ ]:
# =============================================================================
# MAIN EXTRACTION FUNCTION
# =============================================================================

def run_extraction(lines: List[str]) -> pd.DataFrame:
    """Main extraction function with L1/L2/L3/L4 hierarchy."""
    entries = []
    start_idx = find_content_start(lines)
    print(f"Content starts at line {start_idx}")

    hierarchy = HierarchyTracker()

    # Regex patterns for hierarchy and code markers
    hierarchy_patterns = [
        (r'«L1:(.+?)»', hierarchy.set_l1),
        (r'«L2:(.+?)»', hierarchy.set_l2),
        (r'«L3:(.+?)»', hierarchy.set_l3),
        (r'«L4:(.+?)»', hierarchy.set_l4),
    ]
    code_pattern = re.compile(r'«CODE:(~)?(\d{4})(\*)?»')
    block_end_pattern = re.compile(r'«CODE:|«L1:|«L2:|«L3:|«L4:')

    i = start_idx
    while i < len(lines):
        line = lines[i]

        # Check for hierarchy markers
        hierarchy_matched = False
        for pattern, setter in hierarchy_patterns:
            match = re.search(pattern, line)
            if match:
                readable = extract_hierarchy_text(lines, i, match.group(1))
                setter(readable)
                hierarchy_matched = True
                break

        if hierarchy_matched:
            i += 1
            continue

        # Check for CODE marker
        code_match = code_pattern.search(line)
        if code_match:
            is_provisional_in_tag = code_match.group(1) is not None
            code = code_match.group(2)
            is_asterisked_in_tag = code_match.group(3) is not None
            block_start = i

            # Collect block lines until next marker
            block_lines = [line]
            j = i + 1
            while j < len(lines):
                next_line = lines[j]
                if block_end_pattern.search(next_line):
                    break
                block_lines.append(next_line)
                j += 1

            block = '\n'.join(block_lines)
            fee_info = extract_fee_from_block(block)
            is_xref, xref_to = check_cross_reference(block)

            # Only include entries with fee, by-report, or cross-reference
            if fee_info['fee'] is not None or fee_info['is_by_report'] or is_xref:
                is_provisional = is_provisional_in_tag or bool(re.search(r'~\s*' + code, block))
                is_asterisked = is_asterisked_in_tag or bool(re.search(code + r'\*', block))

                l1, l2, l3, l4 = hierarchy.get_current()
                section_code = get_section_code(l1)
                specialty_code, specialty_name = get_specialty_info(l1)
                description = extract_description(block, code)
                code_padded = code.zfill(4)

                display = code_padded
                if is_provisional:
                    display = '~' + display
                if is_asterisked:
                    display = display + '*'

                entries.append({
                    'tariff_code': code_padded,
                    'tariff_code_display': display,
                    'parent_code': None,
                    'section_code': section_code,
                    'section_name': l1,
                    'specialty_code': specialty_code,
                    'specialty_name': specialty_name,
                    'category': l2,
                    'subcategory': l3,
                    'subsubcategory': l4,
                    'description': description,
                    'notes': extract_notes(block),
                    'fee_total': fee_info['fee'],
                    'fee_technical': fee_info['tec'],
                    'fee_professional': fee_info['pro'],
                    'unit_value': fee_info['unit_value'],
                    'is_add_on': is_add_on_fee(block, description),
                    'is_provisional': is_provisional,
                    'is_asterisked': is_asterisked,
                    'is_by_report': fee_info['is_by_report'],
                    'is_cross_reference': is_xref,
                    'cross_reference_to': xref_to,
                    'applicable_rules': extract_rules(block),
                    'time_requirement_minutes': extract_time_requirement(block),
                    'source_line': block_start + 1,
                })

            i = j
            continue

        i += 1

    return pd.DataFrame(entries)


print("✓ Main extraction function defined")

✓ Main extraction function defined


In [ ]:
# =============================================================================
# RUN PHASE 1 EXTRACTION
# =============================================================================

print("=" * 60)
print("PHASE 1: EXTRACTION")
print("=" * 60)

df = run_extraction(LINES)

print(f"\n✓ Extracted {len(df):,} entries")
print(f"  Unique codes: {df['tariff_code'].nunique():,}")
print(f"  Has fee: {df['fee_total'].notna().sum():,} ({100*df['fee_total'].notna().sum()/len(df):.1f}%)")
print(f"  By Report: {df['is_by_report'].sum():,}")
print(f"  Add-on fees: {df['is_add_on'].sum():,}")
print(f"\nHierarchy coverage:")
print(f"  Has L1 (section_name): {(df['section_name'] != '').sum():,}")
print(f"  Has L2 (category): {(df['category'] != '').sum():,}")
print(f"  Has L3 (subcategory): {(df['subcategory'] != '').sum():,}")
print(f"  Has L4 (subsubcategory): {(df['subsubcategory'] != '').sum():,}")

PHASE 1: EXTRACTION
Content starts at line 3382

✓ Extracted 4,607 entries
  Unique codes: 3,782
  Has fee: 4,437 (96.3%)
  By Report: 204
  Add-on fees: 413

Hierarchy coverage:
  Has L1 (section_name): 4,607
  Has L2 (category): 4,511
  Has L3 (subcategory): 1,976
  Has L4 (subsubcategory): 24


In [ ]:
# Show hierarchy examples
print("Sample hierarchy:")
sample = df[df['subcategory'] != ''].head(20)
for idx, row in sample.iterrows():
    print(f"\n{row['tariff_code']}:")
    print(f"  L1: {row['section_name'][:40]}")
    print(f"  L2: {row['category'][:40] if row['category'] else '(none)'}")
    print(f"  L3: {row['subcategory'][:40] if row['subcategory'] else '(none)'}")
    print(f"  Desc: {row['description'][:40]}")

Sample hierarchy:

0171:
  L1: Integumentary System
  L2: Cutaneous Procedures
  L3: Investigation
  Desc: Biopsy of skin, subcutaneous tissue or m

0172:
  L1: Integumentary System
  L2: Cutaneous Procedures
  L3: Investigation
  Desc: Dermatoscopy

0415:
  L1: Integumentary System
  L2: Cutaneous Procedures
  L3: Investigation
  Desc: Woods light examination

0106:
  L1: Integumentary System
  L2: Cutaneous Procedures
  L3: Incision
  Desc: Abscess or hematoma, puncture aspiration

0103:
  L1: Integumentary System
  L2: Cutaneous Procedures
  L3: Incision
  Desc: Carbuncle drainage

0101:
  L1: Integumentary System
  L2: Cutaneous Procedures
  L3: Incision
  Desc: Superficial localized infection such as 

0170:
  L1: Integumentary System
  L2: Cutaneous Procedures
  L3: Incision
  Desc: Acne Surgery-Marsupialization, opening o

0130:
  L1: Integumentary System
  L2: Cutaneous Procedures
  L3: Incision
  Desc: Foreign body subcutaneous tissue, remova

0256:
  L1: Integumentary System


In [ ]:
# Validate known codes
print("\nKnown code validation:")
KNOWN_CODES = [
    ('8540', 112.42),
    ('8550', 184.96),
    ('8403', 59.05),
    ('8563', 54.86),
    ('0770', 518.14),
]

for code, expected_fee in KNOWN_CODES:
    matches = df[df['tariff_code'] == code]
    if len(matches) > 0:
        actual = matches.iloc[0]['fee_total']
        status = '✓' if abs(actual - expected_fee) < 0.01 else f'✗ got {actual}'
        print(f"  {code}: {expected_fee} -> {status}")
    else:
        print(f"  {code}: NOT FOUND")


Known code validation:
  8540: 112.42 -> ✓
  8550: 184.96 -> ✓
  8403: 59.05 -> ✓
  8563: NOT FOUND
  0770: 518.14 -> ✓


In [ ]:
# Apply title case cleaning to text columns
TEXT_COLUMNS = ['section_name', 'category', 'subcategory', 'subsubcategory', 'description']

for col in TEXT_COLUMNS:
    if col in df.columns:
        df[col] = df[col].apply(to_title_case)

print("✓ Data cleaning applied (title case)")
print(f"\nSample cleaned data:")
sample = df[['tariff_code', 'section_name', 'category', 'subcategory', 'description']].head(10)
for idx, row in sample.iterrows():
    category_display = row['category'][:30] if row['category'] else '(none)'
    print(f"  {row['tariff_code']}: {category_display} > {row['description'][:30]}")

✓ Data cleaning applied (title case)

Sample cleaned data:
  8561: (none) > For Special Calls Made to a Pa
  8598: (none) > For Special Calls Made to the 
  8566: (none) > For Special Calls Made in Obst
  8567: (none) > For Special Calls Made in Non-
  8645: Office , Home Visits > Extended Complete History and 
  8540: Office , Home Visits > Complete History and Physical 
  8646: Office , Home Visits > Extended Complete or Extensive
  8502: Office , Home Visits > Complete or Extensive Re-Exami
  8647: Office , Home Visits > Extended Regional History & Ex
  8403: Office , Home Visits > Regional History and Examinati


In [ ]:
# =============================================================================
# EXPORT UTILITIES
# =============================================================================

def prefix_for_excel(value, prefix="'"):
    """Prefix a value to prevent Excel auto-formatting (e.g., dropping leading zeros)."""
    if pd.isna(value) or value == '' or value is None:
        return value
    return f"{prefix}{value}"


def clean_tariff_code(x):
    """Clean and format a single tariff code (4 digits, no .0)."""
    if pd.isna(x) or x == '' or x is None:
        return None
    return str(x).replace('.0', '').strip().zfill(4)


def clean_tariff_code_list(x):
    """Clean and format a comma-separated list of tariff codes."""
    if pd.isna(x) or x == '' or x is None:
        return None
    codes = [c.strip().replace('.0', '').zfill(4) for c in str(x).split(',')]
    return ', '.join(codes)


print("✓ Export utilities defined")

✓ Export utilities defined


In [ ]:
# Export Phase 1 output
PHASE1_FILE = 'mb_tariffs_phase1.csv'

df_phase1 = df.copy()
df_phase1['tariff_code'] = df_phase1['tariff_code'].apply(lambda x: prefix_for_excel(x))
df_phase1['specialty_code'] = df_phase1['specialty_code'].apply(
    lambda x: prefix_for_excel(x) if x != '' else x
)

df_phase1.to_csv(PHASE1_FILE, index=False, encoding='utf-8')
print(f"✓ Phase 1 export saved: {PHASE1_FILE}")
print(f"  {len(df)} rows, {len(df.columns)} columns")
files.download(PHASE1_FILE)

✓ Phase 1 export saved: mb_tariffs_phase1.csv
  4607 rows, 25 columns


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---
## Phase 2: GPT Full Review

GPT reviews ALL entries for:
- Description completion (if L1+L2+L3+L4+desc insufficient)
- Parent code relationships
- Add-on fee detection (is_add_on, add_on_to)
- Age restrictions
- Setting restrictions
- Exclusions

---

In [ ]:
# Setup OpenAI (secure API key input - key will NOT be displayed)
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print("✓ API key loaded from Colab secrets")
except:
    OPENAI_API_KEY = getpass.getpass("Enter OpenAI API key (input hidden): ")
    print("✓ API key entered")

client = OpenAI(api_key=OPENAI_API_KEY)
print("✓ OpenAI client ready")

Enter OpenAI API key (input hidden): ··········
✓ API key entered
✓ OpenAI client ready


In [ ]:
# Phase 2 cost and time estimates
total_entries = len(df)
estimated_cost = total_entries * COST_PER_ENTRY_ESTIMATE
estimated_minutes = total_entries * TIME_PER_ENTRY_SECONDS / 60

print(f"Total entries to review: {total_entries}")
print(f"Estimated cost: ~${estimated_cost:.2f}")
print(f"Estimated time: ~{estimated_minutes:.0f} minutes")

Total entries to review: 4607
Estimated cost: ~$23.04
Estimated time: ~38 minutes


In [ ]:
# =============================================================================
# GPT REVIEW SYSTEM PROMPT
# =============================================================================

SYSTEM_PROMPT = '''You are an expert medical billing coder reviewing Manitoba's Physician's Manual.

For EACH code, extract the following:

═══════════════════════════════════════════════════════════════
1. DESCRIPTION COMPLETION
═══════════════════════════════════════════════════════════════
IMPORTANT: L1, L2, L3, L4 are ALREADY stored in separate columns.
Do NOT repeat them in the description.

If a description seems incomplete (e.g., "open reduction", "and left heart cath"):
- Set parent_code to link it to its parent
- KEEP the description SHORT as-is - the parent_code provides context
- Users can follow the parent_code chain to get full meaning

ONLY expand a description if there is NO parent AND it is truly ambiguous alone.
Set needs_completion=false in most cases. Keep original descriptions.

═══════════════════════════════════════════════════════════════
2. PARENT_CODE
═══════════════════════════════════════════════════════════════

WHY THIS MATTERS:
This dataset must be usable standalone. Every row must make sense on its own.
Right now, "Open Reduction" appears dozens of times with no body part.
That's useless - open reduction of WHAT? We need parent_code to link related
procedures so users can understand what each code actually means.

WHAT IS A CHILD CODE:
A child code is a code whose description is incomplete on its own.
The Manitoba manual uses INDENTATION to avoid repeating anatomy.
"Open reduction" indented under "Femur, neck" means femur neck open reduction.
The child inherits meaning from its parent.

HOW TO IDENTIFY CHILD CODES:
Look at the WHITESPACE before descriptions in the source context.
More indentation = child of something above it.
The parent is the FIRST preceding code with LESS indentation - then STOP.

SET parent_code TO THE IMMEDIATE PARENT:
    0865    Femur, neck, closed reduction              → parent_code: null
    0868       open reduction                          → parent_code: "0865"
    0870          prosthetic replacement               → parent_code: "0868"
    0877       slipped upper femoral epiphysis         → parent_code: "0865"
    0872       intertrochanteric, closed reduction     → parent_code: "0865"
    0874          open reduction                       → parent_code: "0872"

Notice: 0877 and 0872 return to the SAME indentation level as 0868, so they
share the same parent (0865). 0874 is indented under 0872, so its parent is 0872.

This creates a chain: grandchild → child → parent. For example:
- 0870's full meaning: Femur neck → open reduction → prosthetic replacement
- 0874's full meaning: Femur → intertrochanteric → open reduction

CRITICAL - STOP AT FIRST MATCH:
    3872    Percutaneous nephrostomy...                → parent_code: null
    3873    Single stone removal without lithotripsy   → parent_code: null (same level)
    3875       plus nephrostomy...                     → parent_code: "3873"
    3878       with electrohydraulic lithotripsy...    → parent_code: "3873" (NOT 3872!)
    3879          plus nephrostomy...                  → parent_code: "3878"

For 3878: scan backwards, skip 3875 (more indent), hit 3873 (less indent) → STOP.
Do NOT continue back to 3872 even though it has the same indent as 3873.

ALSO USE parent_code FOR TIERED/VARIANT CODES:
- "Day 2-10, per day" → parent is "Day 1" code
- "Each additional 15 minutes" → parent is base time code
- Level B care → parent is Level A code
- Any tiered/variant where you pick ONE based on situation

The parent_code must be a 4-digit code VISIBLE in the nearby context.

═══════════════════════════════════════════════════════════════
3. ADD-ON DETECTION (for SUPPLEMENTS - billed TOGETHER)
═══════════════════════════════════════════════════════════════
is_add_on = true ONLY when THIS CODE is explicitly a supplement
that gets billed ON TOP OF another service.

THE ", ADD" RULE - MOST IMPORTANT:
If the description ends with ", Add" → this is an ADD-ON, not a child code.
Set is_add_on=true and parent_code=null.
Examples: "Intracoronary Drug Injection, Add" → is_add_on=true, parent_code=null
          "Coronary Thrombectomy, Add" → is_add_on=true, parent_code=null

DEFINITE ADD-ONS (is_add_on: true):
- Description ends with ", Add" or ", add" ← THIS IS THE KEY INDICATOR
- Description contains "supplement add to" or "add to [X] fee"
- Code is a percentage fee (20%, 25%) added to another service
- Notes say THIS code "may be claimed in addition to" other codes
- Special Call fees (8561, 8598, 8566, 8567, 8563) - these are add-ons

NOT ADD-ONS (is_add_on: false):
- "premium will be applied to tariffs X, Y, Z" - this describes a MODIFIER, not an add-on code
- Lists of tariff codes that receive a premium - the listed codes are NOT add-ons
- Regular examination/visit codes (8540, 8550, 8645, etc.)
- "Each additional X" patterns - these are VARIANTS, use parent_code instead
- Codes where OTHER things can be claimed with them - that doesn't make THIS code an add-on

CRITICAL DISTINCTION:
- "8700 supplement add to visit fee" → 8700 IS an add-on
- "premium applied to 8540 in hospital" → 8540 is NOT an add-on (it receives a premium)

add_on_to: Only fill if specific codes are stated that THIS code adds to.
Leave null if it says generic "visit fee" or "surgical fee".

═══════════════════════════════════════════════════════════════
4. AGE RESTRICTION (only if EXPLICITLY stated)
═══════════════════════════════════════════════════════════════
Look for explicit age requirements in the notes:
- "under 18", "under eighteen", "child", "pediatric"
- "over 65", "elderly"
- "newborn", "infant", "neonate"

Return the exact restriction text. null if not mentioned.

═══════════════════════════════════════════════════════════════
5. SETTING RESTRICTION (only if EXPLICITLY stated)
═══════════════════════════════════════════════════════════════
Look for explicit setting requirements:
- "hospital only", "hospital in-patient"
- "office", "home"
- "ICU", "NICU", "PICU"
- "Emergency Department"

Return the exact restriction text. null if not mentioned.

═══════════════════════════════════════════════════════════════
6. EXCLUSIONS (only if EXPLICITLY stated)
═══════════════════════════════════════════════════════════════
Look for explicit billing exclusions:
- "cannot be claimed with", "may not be claimed with"
- "not claimable with", "excludes"
- "does not include"

Return the exact exclusion text. null if not mentioned.

═══════════════════════════════════════════════════════════════
CRITICAL RULES
═══════════════════════════════════════════════════════════════
1. Do NOT invent data. Only extract what is EXPLICITLY stated or clearly visible.

2. ", ADD" SUFFIX = ADD-ON (MUTUALLY EXCLUSIVE WITH PARENT_CODE):
   If description ends with ", Add" → is_add_on=true, parent_code=null
   These are NEVER child codes. They are billed ON TOP OF other procedures.

3. INDENTED = CHILD CODE (uses parent_code, NOT is_add_on):
   If code is indented under another → parent_code=that code, is_add_on=false
   These inherit context from their parent.

4. VARIANT = CHILD CODE (uses parent_code):
   "Day 2-10" under "Day 1" → parent_code links them, is_add_on=false
   You pick ONE based on situation, not both.

5. Do NOT expand descriptions with L1/L2/L3 - those are in separate columns.
   Keep descriptions short. The parent_code chain provides context.

6. When you see a list of tariff codes in context, do NOT assume they are
   related to the current code unless explicitly stated.

7. "Premium applied to X" means X receives a premium, NOT that X is an add-on.

8. READ THE INDENTATION CAREFULLY. This is how the manual groups related procedures.

Return JSON:
{
  "needs_completion": true/false,
  "description": "...",
  "parent_code": "XXXX" or null,
  "is_add_on": true/false,
  "add_on_to": "XXXX, YYYY" or null,
  "age_restriction": "..." or null,
  "setting_restriction": "..." or null,
  "exclusions": "..." or null
}
'''

print("✓ System prompt defined")

✓ System prompt defined


In [ ]:
# =============================================================================
# GPT EVALUATION FUNCTIONS
# =============================================================================

def get_source_context(source_line: int, lines: List[str], before: int = 30, after: int = 5) -> str:
    """Get surrounding context from source file for GPT review."""
    if pd.isna(source_line) or source_line < 1:
        return "(no source context available)"
    source_line = int(source_line)
    start = max(0, source_line - before - 1)
    end = min(len(lines), source_line + after)
    context = '\n'.join(lines[start:end])
    context = context.replace('\f', '\n').strip()
    return context[:MAX_CONTEXT_LENGTH]


def build_user_message(row: pd.Series, context: str) -> str:
    """Build the user message for GPT review."""
    return f"""Code: {row['tariff_code']}

HIERARCHY:
  L1 (section): {row['section_name']}
  L2 (category): {row['category'] if row['category'] else '(none)'}
  L3 (subcategory): {row['subcategory'] if row['subcategory'] else '(none)'}
  L4 (subsubcategory): {row['subsubcategory'] if row['subsubcategory'] else '(none)'}

Description: "{row['description']}"
Notes: "{row['notes'] if row['notes'] else '(none)'}"

Source context:
```
{context}
```"""


def parse_gpt_response(result: Dict, original_description: str) -> Dict:
    """Parse and clean GPT response."""
    # Clean description
    desc = result.get('description', original_description)
    desc = clean_text(desc)
    if len(desc) > MAX_DESCRIPTION_LENGTH:
        desc = desc[:MAX_DESCRIPTION_LENGTH - 3] + '...'

    # Clean parent_code
    parent = result.get('parent_code')
    if parent:
        parent = str(parent).replace('.0', '').strip().zfill(4)
        if not re.match(r'^\d{4}$', parent):
            parent = None

    # Clean add_on_to (can be single code or comma-separated list)
    add_on_to = result.get('add_on_to')
    if add_on_to:
        add_on_to = str(add_on_to).strip()
        codes = [c.strip().replace('.0', '').zfill(4) for c in add_on_to.split(',')]
        codes = [c for c in codes if re.match(r'^\d{4}$', c)]
        add_on_to = ', '.join(codes) if codes else None

    return {
        'needs_completion': result.get('needs_completion', False),
        'description': desc,
        'parent_code': parent,
        'is_add_on': result.get('is_add_on', False),
        'add_on_to': add_on_to,
        'age_restriction': result.get('age_restriction'),
        'setting_restriction': result.get('setting_restriction'),
        'exclusions': result.get('exclusions')
    }


def get_empty_result(original_description: str) -> Dict:
    """Return empty result for error cases."""
    return {
        'needs_completion': False,
        'description': original_description,
        'parent_code': None,
        'is_add_on': False,
        'add_on_to': None,
        'age_restriction': None,
        'setting_restriction': None,
        'exclusions': None
    }


def evaluate_entry(row: pd.Series, lines: List[str]) -> Dict:
    """Send entry to GPT for full evaluation."""
    context = get_source_context(row['source_line'], lines)
    user_msg = build_user_message(row, context)

    try:
        response = client.chat.completions.create(
            model=GPT_MODEL,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_msg}
            ],
            temperature=GPT_TEMPERATURE,
            response_format={"type": "json_object"}
        )
        result = json.loads(response.choices[0].message.content)
        return parse_gpt_response(result, row['description'])
    except Exception as e:
        print(f"  Error on {row['tariff_code']}: {e}")
        return get_empty_result(row['description'])


print("✓ GPT evaluation functions defined")

✓ GPT evaluation functions defined


In [ ]:
# =============================================================================
# CHECKPOINTING FUNCTIONS
# =============================================================================

def save_checkpoint(results: Dict, stats: Dict, last_processed_idx: int, checkpoint_file: str = CHECKPOINT_FILE):
    """Save current progress to a checkpoint file."""
    checkpoint_data = {
        'results': {str(k): v for k, v in results.items()},  # Convert keys to strings for JSON
        'stats': stats,
        'last_processed_idx': last_processed_idx,
        'timestamp': time.strftime('%Y-%m-%d %H:%M:%S')
    }
    with open(checkpoint_file, 'w', encoding='utf-8') as f:
        json.dump(checkpoint_data, f, indent=2)
    print(f"  [Checkpoint saved: {last_processed_idx + 1} entries]")


def load_checkpoint(checkpoint_file: str = CHECKPOINT_FILE) -> Tuple[Dict, Dict, int]:
    """Load progress from a checkpoint file if it exists."""
    if not os.path.exists(checkpoint_file):
        return {}, {'completed': 0, 'parents': 0, 'add_ons': 0, 'age_restricted': 0, 'setting_restricted': 0, 'has_exclusions': 0}, -1

    try:
        with open(checkpoint_file, 'r', encoding='utf-8') as f:
            checkpoint_data = json.load(f)

        # Convert string keys back to integers
        results = {int(k): v for k, v in checkpoint_data['results'].items()}
        stats = checkpoint_data['stats']
        last_processed_idx = checkpoint_data['last_processed_idx']

        print(f"✓ Loaded checkpoint from {checkpoint_data['timestamp']}")
        print(f"  Resuming from entry {last_processed_idx + 1}")
        print(f"  Stats so far: completions={stats['completed']}, parents={stats['parents']}, add-ons={stats['add_ons']}")

        return results, stats, last_processed_idx
    except Exception as e:
        print(f"Warning: Could not load checkpoint ({e}). Starting fresh.")
        return {}, {'completed': 0, 'parents': 0, 'add_ons': 0, 'age_restricted': 0, 'setting_restricted': 0, 'has_exclusions': 0}, -1


def clear_checkpoint(checkpoint_file: str = CHECKPOINT_FILE):
    """Remove checkpoint file after successful completion."""
    if os.path.exists(checkpoint_file):
        os.remove(checkpoint_file)
        print(f"✓ Checkpoint file removed")


print("✓ Checkpointing functions defined")

✓ Checkpointing functions defined


In [ ]:
# =============================================================================
# TEST CONFIGURATION
# =============================================================================
# Set TEST_MODE = True to test on a subset of codes
# Use the filters below to target specific sections for testing

TEST_MODE = False  # Set to False for full run

# --- FILTER OPTIONS (only used when TEST_MODE = True) ---

# Option 1: Filter by L2 category (e.g., "Lower Extremity", "Upper Extremity")
TEST_CATEGORY = None  # Set to None to disable

# Option 2: Filter by L1 section name (partial match)
TEST_SECTION = None  # e.g., "Musculoskeletal" - Set to None to disable

# Option 3: Filter by tariff code range
TEST_CODE_START = None  # e.g., "0865" - Set to None to disable
TEST_CODE_END = None    # e.g., "0930" - Set to None to disable

# Option 4: Filter by specific codes (list)
TEST_CODES = None  # e.g., ["0865", "0868", "0870", "0872", "0874"] - Set to None to disable

# Option 5: Just take first N entries (fallback)
TEST_LIMIT = 50

# --- BUILD TEST DATAFRAME ---

if TEST_MODE:
    process_df = df.copy()
    filter_applied = False

    # Apply category filter
    if TEST_CATEGORY:
        process_df = process_df[process_df['category'].str.contains(TEST_CATEGORY, case=False, na=False)]
        print(f"✓ Filtered by category '{TEST_CATEGORY}': {len(process_df)} entries")
        filter_applied = True

    # Apply section filter
    elif TEST_SECTION:
        process_df = process_df[process_df['section_name'].str.contains(TEST_SECTION, case=False, na=False)]
        print(f"✓ Filtered by section '{TEST_SECTION}': {len(process_df)} entries")
        filter_applied = True

    # Apply code range filter
    elif TEST_CODE_START and TEST_CODE_END:
        process_df = process_df[(process_df['tariff_code'] >= TEST_CODE_START) &
                                 (process_df['tariff_code'] <= TEST_CODE_END)]
        print(f"✓ Filtered by code range {TEST_CODE_START}-{TEST_CODE_END}: {len(process_df)} entries")
        filter_applied = True

    # Apply specific codes filter
    elif TEST_CODES:
        process_df = process_df[process_df['tariff_code'].isin(TEST_CODES)]
        print(f"✓ Filtered by specific codes {TEST_CODES}: {len(process_df)} entries")
        filter_applied = True

    # Fallback to limit
    if not filter_applied or len(process_df) == 0:
        process_df = df.head(TEST_LIMIT).copy()
        print(f"✓ TEST MODE: First {len(process_df)} entries")

    # Show what we're testing
    print(f"\nCodes to process:")
    for idx, row in process_df.head(20).iterrows():
        print(f"  {row['tariff_code']}: {row['description'][:50]}")
    if len(process_df) > 20:
        print(f"  ... and {len(process_df) - 20} more")
else:
    process_df = df.copy()
    print(f"FULL RUN: Processing all {len(process_df)} entries")

print(f"\nEstimated time: ~{len(process_df) * TIME_PER_ENTRY_SECONDS / 60:.1f} minutes")
print(f"Estimated cost: ~${len(process_df) * COST_PER_ENTRY_ESTIMATE:.2f}")

FULL RUN: Processing all 4607 entries

Estimated time: ~38.4 minutes
Estimated cost: ~$23.04


In [ ]:
# =============================================================================
# RUN PHASE 2 GPT REVIEW (with checkpointing)
# =============================================================================

print("\n" + "=" * 60)
print("PHASE 2: GPT FULL REVIEW")
print("=" * 60 + "\n")

# Load checkpoint if exists (allows resuming after crash)
results, stats, last_processed_idx = load_checkpoint()

# Build list of (position, index, row) to process
entries_to_process = list(enumerate(process_df.iterrows()))
start_position = last_processed_idx + 1

if start_position > 0:
    print(f"Skipping {start_position} already-processed entries...")

for i, (idx, row) in entries_to_process[start_position:]:
    # Progress reporting
    if (i + 1) % PROGRESS_REPORT_INTERVAL == 0:
        print(f"  {i+1}/{len(process_df)} - completions:{stats['completed']}, parents:{stats['parents']}, add-ons:{stats['add_ons']}")

    # Evaluate entry
    result = evaluate_entry(row, LINES)
    results[idx] = result

    # Update statistics
    if result['needs_completion']:
        stats['completed'] += 1
    if result['parent_code']:
        stats['parents'] += 1
    if result['is_add_on']:
        stats['add_ons'] += 1
    if result['age_restriction']:
        stats['age_restricted'] += 1
    if result['setting_restriction']:
        stats['setting_restricted'] += 1
    if result['exclusions']:
        stats['has_exclusions'] += 1

    # Save checkpoint periodically
    if (i + 1) % CHECKPOINT_INTERVAL == 0:
        save_checkpoint(results, stats, i)

    time.sleep(API_CALL_DELAY_SECONDS)

# Final checkpoint save
save_checkpoint(results, stats, len(process_df) - 1)

print(f"\n✓ Phase 2 complete!")
print(f"  Descriptions completed: {stats['completed']}")
print(f"  Parent codes: {stats['parents']}")
print(f"  Add-on fees: {stats['add_ons']}")
print(f"  Age restricted: {stats['age_restricted']}")
print(f"  Setting restricted: {stats['setting_restricted']}")
print(f"  Has exclusions: {stats['has_exclusions']}")


PHASE 2: GPT FULL REVIEW

✓ Loaded checkpoint from 2026-01-28 23:37:08
  Resuming from entry 4607
  Stats so far: completions=1, parents=928, add-ons=322
Skipping 4607 already-processed entries...
  [Checkpoint saved: 4607 entries]

✓ Phase 2 complete!
  Descriptions completed: 1
  Parent codes: 928
  Add-on fees: 322
  Age restricted: 162
  Setting restricted: 169
  Has exclusions: 386


In [ ]:
# =============================================================================
# APPLY GPT RESULTS TO DATAFRAME
# =============================================================================

# Add new columns
df['_desc_original'] = df['description']
df['add_on_to'] = None
df['age_restriction'] = None
df['setting_restriction'] = None
df['exclusions'] = None

# Apply results
for idx, result in results.items():
    if result['needs_completion']:
        df.loc[idx, 'description'] = result['description']
    if result['parent_code']:
        df.loc[idx, 'parent_code'] = result['parent_code']
    # GPT-determined is_add_on overrides parser
    df.loc[idx, 'is_add_on'] = result['is_add_on']
    if result['add_on_to']:
        df.loc[idx, 'add_on_to'] = result['add_on_to']
    if result['age_restriction']:
        df.loc[idx, 'age_restriction'] = result['age_restriction']
    if result['setting_restriction']:
        df.loc[idx, 'setting_restriction'] = result['setting_restriction']
    if result['exclusions']:
        df.loc[idx, 'exclusions'] = result['exclusions']

print(f"Applied {stats['completed']} description updates")
print(f"Applied {stats['parents']} parent codes")
print(f"Applied {stats['add_ons']} add-on flags")

Applied 1 description updates
Applied 928 parent codes
Applied 322 add-on flags


In [ ]:
# =============================================================================
# DISPLAY SAMPLE RESULTS
# =============================================================================

def display_sample_results(results: Dict, df: pd.DataFrame, category: str,
                           filter_fn, format_fn, limit: int = 8):
    """Display sample results for a given category."""
    print(f"\n--- {category} ---")
    items = [(idx, r) for idx, r in results.items() if filter_fn(r)][:limit]
    for idx, result in items:
        row = df.loc[idx]
        print(format_fn(row, result))


print("\nSample results:")

# Description completions
display_sample_results(
    results, df, "DESCRIPTION COMPLETIONS",
    lambda r: r['needs_completion'],
    lambda row, r: (
        f"  {row['tariff_code']}:{' [parent: ' + str(row['parent_code']) + ']' if pd.notna(row['parent_code']) else ''}\n"
        f"    Before: '{row['_desc_original']}'\n"
        f"    After:  '{row['description']}'"
    )
)

# Parent codes
display_sample_results(
    results, df, "PARENT CODES",
    lambda r: r['parent_code'],
    lambda row, r: f"  {row['tariff_code']}: {row['description'][:40]} -> parent: {r['parent_code']}"
)

# Add-on fees
display_sample_results(
    results, df, "ADD-ON FEES",
    lambda r: r['is_add_on'],
    lambda row, r: (
        f"  {row['tariff_code']}: {row['description'][:40]}"
        f"{' -> adds to ' + r['add_on_to'] if r['add_on_to'] else ''}"
    )
)

# Age restrictions
display_sample_results(
    results, df, "AGE RESTRICTIONS",
    lambda r: r['age_restriction'],
    lambda row, r: f"  {row['tariff_code']}: {r['age_restriction']}"
)

# Setting restrictions
display_sample_results(
    results, df, "SETTING RESTRICTIONS",
    lambda r: r['setting_restriction'],
    lambda row, r: f"  {row['tariff_code']}: {r['setting_restriction']}"
)

# Exclusions
display_sample_results(
    results, df, "EXCLUSIONS",
    lambda r: r['exclusions'],
    lambda row, r: f"  {row['tariff_code']}: {r['exclusions'][:60]}"
)


Sample results:

--- DESCRIPTION COMPLETIONS ---
  8685:
    Before: 'Single Dose'
    After:  'Single Dose'

--- PARENT CODES ---
  8595: Consultation-Unassigned Patient -> parent: 8550
  8404: Complete or Extensive Re-Assessment and  -> parent: 8552
  8558: Behaviour Therapy Conducted Subsequent t -> parent: 8552
  8301: Day 2 - 10, Per Day -> parent: 8300
  8302: Day 11 Onwards, Per Day -> parent: 8300
  8304: Day 2 - 10, Per Day -> parent: 8303
  8305: Day 11 Onwards, Per Day -> parent: 8303
  8307: Day 2 - 10, Per Day -> parent: 8306

--- ADD-ON FEES ---
  8561: For Special Calls Made to a Patient’S Ho
  8598: For Special Calls Made to the Emergency 
  8566: For Special Calls Made in Obstetrics
  8567: For Special Calls Made in Non-Elective S
  8700: Continuing Patient Care Management, Supp
  8526: Clinical Teaching Unit (Ctu) Patient Car -> adds to 8520
  8700: Continuing Patient Care Management, Supp
  8614: Interpretation of Comprehensive Cognitiv

--- AGE RESTRICTIONS ---
  8

In [ ]:
# =============================================================================
# FINAL CLEANUP AND COLUMN ORDERING
# =============================================================================

# Drop temporary columns
drop_cols = ['_desc_original']
df_final = df.drop(columns=[c for c in drop_cols if c in df.columns], errors='ignore')

# Reorder columns to match output spec
df_final = df_final[[c for c in OUTPUT_COLUMNS if c in df_final.columns]]

print(f"Final dataset: {len(df_final)} rows x {len(df_final.columns)} columns")

Final dataset: 4607 rows x 28 columns


In [ ]:
# =============================================================================
# FINAL SUMMARY
# =============================================================================

print("\n" + "=" * 60)
print("FINAL SUMMARY")
print("=" * 60)

print(f"\nTotal entries: {len(df_final):,}")
print(f"Unique codes: {df_final['tariff_code'].nunique():,}")

print(f"\nFee coverage:")
print(f"  Has fee: {df_final['fee_total'].notna().sum():,} ({100*df_final['fee_total'].notna().sum()/len(df_final):.1f}%)")
print(f"  By Report: {df_final['is_by_report'].sum():,}")
print(f"  TEC/PRO: {df_final['fee_technical'].notna().sum():,}")

print(f"\nHierarchy:")
print(f"  Has category (L2): {(df_final['category'] != '').sum():,}")
print(f"  Has subcategory (L3): {(df_final['subcategory'] != '').sum():,}")
print(f"  Has subsubcategory (L4): {(df_final['subsubcategory'] != '').sum():,}")

print(f"\nGPT Review:")
print(f"  Entries reviewed: {len(results)}")
print(f"  Descriptions completed: {stats['completed']}")
print(f"  Parent codes: {df_final['parent_code'].notna().sum():,}")
print(f"  Add-on fees: {df_final['is_add_on'].sum():,}")
print(f"  Age restrictions: {df_final['age_restriction'].notna().sum():,}")
print(f"  Setting restrictions: {df_final['setting_restriction'].notna().sum():,}")
print(f"  Exclusions: {df_final['exclusions'].notna().sum():,}")


FINAL SUMMARY

Total entries: 4,607
Unique codes: 3,782

Fee coverage:
  Has fee: 4,437 (96.3%)
  By Report: 204
  TEC/PRO: 248

Hierarchy:
  Has category (L2): 4,511
  Has subcategory (L3): 1,976
  Has subsubcategory (L4): 24

GPT Review:
  Entries reviewed: 4607
  Descriptions completed: 1
  Parent codes: 928
  Add-on fees: 322
  Age restrictions: 162
  Setting restrictions: 169
  Exclusions: 386


In [ ]:
# =============================================================================
# EXPORT FINAL OUTPUT
# =============================================================================

OUTPUT_FILE = 'mb_tariffs_enriched.csv'

df_export = df_final.copy()

# Clean code fields
df_export['parent_code'] = df_export['parent_code'].apply(clean_tariff_code)
df_export['add_on_to'] = df_export['add_on_to'].apply(clean_tariff_code_list)

# Prefix codes for Excel compatibility
df_export['tariff_code'] = df_export['tariff_code'].apply(prefix_for_excel)
df_export['specialty_code'] = df_export['specialty_code'].apply(
    lambda x: prefix_for_excel(x) if pd.notna(x) and x != '' else x
)

df_export.to_csv(OUTPUT_FILE, index=False, encoding='utf-8')
print(f"✓ Saved {OUTPUT_FILE}")

# Clear checkpoint after successful export
clear_checkpoint()

files.download(OUTPUT_FILE)

✓ Saved mb_tariffs_enriched.csv
✓ Checkpoint file removed


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>